In [2]:
import pandas as pd
from pprint import pprint
data_file_path = "Austin Bicycle Crashes 2010-2017.csv"
dataset = pd.read_csv(data_file_path)
dataset.head()

,Crash ID,$1000 Damage to Any One Person's Property,Active School Zone Flag,Adjusted Average Daily Traffic Amount,Agency,At Intersection Flag,Average Daily Traffic Amount,Average Daily Traffic Year,Case ID,City,...,Roadway Relation,Roadway Type,Speed Limit,Street Name,Street Number,Surface Condition,Traffic Control Type,Weather Condition,Person Helmet,Person Type
0,11243623,No,No,15262,Austin Police Department,False,15262,2014,100010723,Austin,...,On Roadway,"4 Or More Lanes, Undivided",45,FM0969,4716,Dry,Marked Lanes,Clear,"Worn, Damaged",Pedalcyclist
1,12112069,No,No,No Data,Austin Police Department,False,No Data,No Data,100010834,Austin,...,On Roadway,No Data,35,E RIVERSIDE DR,200,Dry,Center Stripe/Divider,Clear,Not Worn,Pedalcyclist
2,12112069,No,No,No Data,Austin Police Department,False,No Data,No Data,100010834,Austin,...,On Roadway,No Data,35,E RIVERSIDE DR,200,Dry,Center Stripe/Divider,Clear,Not Worn,Pedalcyclist
3,12112069,No,No,No Data,Austin Police Department,False,No Data,No Data,100010834,Austin,...,On Roadway,No Data,35,E RIVERSIDE DR,200,Dry,Center Stripe/Divider,Clear,Not Worn,Pedalcyclist
4,11376123,No,No,22775,Austin Police Department,False,22775,2014,100042009,Austin,...,On Roadway,"4 Or More Lanes, Undivided",45,SL0111,1487,Dry,None,Clear,Not Worn,Pedalcyclist


In [3]:
dataset.iloc[0]

Crash ID                                                               11243623
$1000 Damage to Any One Person's Property                                    No
Active School Zone Flag                                                      No
Adjusted Average Daily Traffic Amount                                     15262
Agency                                                 Austin Police Department
At Intersection Flag                                                      False
Average Daily Traffic Amount                                              15262
Average Daily Traffic Year                                                 2014
Case ID                                                               100010723
City                                                                     Austin
Construction Zone Flag                                                       No
Construction Zone Workers Present Flag                                       No
County                                  

In [4]:
dataset.dtypes

Crash ID                                      int64
$1000 Damage to Any One Person's Property    object
Active School Zone Flag                      object
Adjusted Average Daily Traffic Amount        object
Agency                                       object
At Intersection Flag                           bool
Average Daily Traffic Amount                 object
Average Daily Traffic Year                   object
Case ID                                      object
City                                         object
Construction Zone Flag                       object
Construction Zone Workers Present Flag       object
County                                       object
Crash Death Count                             int64
Crash Incapacitating Injury Count             int64
Crash Non-incapacitating Injury Count         int64
Crash Not Injured Count                       int64
Crash Possible Injury Count                   int64
Crash Severity                               object
Crash Time  

In [6]:
# store the fields in another csv file
li = list(dataset.columns)
df = pd.DataFrame(li, columns=["fields"])
f = open("fields before grouping.csv", "w")
f.write(df.to_csv(index=False, line_terminator="\n"))
f.close()

In [7]:
file_path = "data fields grouping.csv"
super_grp = pd.read_csv(file_path)
super_grp = {grp_name: super_grp[grp_name].dropna() for grp_name in super_grp}
super_grp

{'crash_report': 0                             Crash Death Count
 1             Crash Incapacitating Injury Count
 2         Crash Non-incapacitating Injury Count
 3                       Crash Not Injured Count
 4                   Crash Possible Injury Count
 5                                Crash Severity
 6                                    Crash Time
 7                      Crash Total Injury Count
 8                    Crash Unknown Injury Count
 9                                    Crash Year
 10                           Physical Feature 1
 11                             Property Damages
 12                          Manner of Collision
 13                                 Other Factor
 14    $1000 Damage to Any One Person's Property
 15                                     Crash ID
 16                                       Agency
 17                          First Harmful Event
 18                                      Case ID
 Name: crash_report, dtype: object,
 'person': 0    P